In [1]:
import pandas as pd
import numpy as np
import scipy
import sklearn 
import sklearn.datasets
import sklearn.metrics as metrics 
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import StratifiedKFold
from sklearn.svm import SVC
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.feature_selection import mutual_info_classif
from scipy import stats
from datetime import datetime
import matplotlib.pyplot as plt 
import seaborn as sns
from numpy import nan
import re
from scipy import stats

C:\Anaconda\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Anaconda\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
input = pd.read_excel("finalSheet.xlsx")

In [103]:
df = input.copy()

In [104]:
def is_outlier(points, thresh=3.5):
    """
    Returns a boolean array with True if points are outliers and False 
    otherwise.

    Parameters:
    -----------
        points : An numobservations by numdimensions array of observations
        thresh : The modified z-score to use as a threshold. Observations with
            a modified z-score (based on the median absolute deviation) greater
            than this value will be classified as outliers.

    Returns:
    --------
        mask : A numobservations-length boolean array.

    References:
    ----------
        Boris Iglewicz and David Hoaglin (1993), "Volume 16: How to Detect and
        Handle Outliers", The ASQC Basic References in Quality Control:
        Statistical Techniques, Edward F. Mykytka, Ph.D., Editor. 
    """
    if len(points.shape) == 1:
        points = points[:,None]
    median = np.median(points, axis=0)
    diff = np.sum((points - median)**2, axis=-1)
    diff = np.sqrt(diff)
    med_abs_deviation = np.median(diff)

    modified_z_score = 0.6745 * diff / med_abs_deviation

    return modified_z_score > thresh

In [105]:
norm_wage = pd.DataFrame(df.groupby(['Ind_code','Wage_level'])['Year_wage_pw'].median())
norm_wage_off = pd.DataFrame(df.groupby(['Ind_code','Wage_level'])['Year_wage_offer'].median())
norm_wage.columns= ["Median_pw"]
norm_wage_off.columns= ["Median_offer"]


In [106]:
#Getting rid of the outliers for the wages
df['PW_outlier'] = is_outlier(df['Year_wage_pw'])
df['Offer_outlier'] = is_outlier(df['Year_wage_offer'])

df.Year_wage_pw.loc[df['PW_outlier'] == True] = (df.loc[df['PW_outlier'] == True].merge(norm_wage, how = "left", left_on = ['Ind_code','Wage_level'], right_index = True))['Median_pw']
df.Year_wage_offer.loc[df['Offer_outlier'] == True] = (df.loc[df['Offer_outlier'] == True].merge(norm_wage_off, how = "left", left_on = ['Ind_code','Wage_level'], right_index = True))['Median_offer']

C:\Anaconda\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [107]:
df['EducationNA'] = np.where(df['FOREIGN_WORKER_INFO_EDUCATION']==0, 1, 0)
df['Education1'] = np.where(df['FOREIGN_WORKER_INFO_EDUCATION']==1, 1, 0)
df['Education2'] = np.where(df['FOREIGN_WORKER_INFO_EDUCATION']==2, 1, 0)
df['Education3'] = np.where(df['FOREIGN_WORKER_INFO_EDUCATION']==3, 1, 0)
df['Education4'] = np.where(df['FOREIGN_WORKER_INFO_EDUCATION']==4, 1, 0)
df['Education5'] = np.where(df['FOREIGN_WORKER_INFO_EDUCATION']==5, 1, 0)
df['Education6'] = np.where(df['FOREIGN_WORKER_INFO_EDUCATION']==6, 1, 0)
df['CitizenRegion1'] = np.where(df['CitizenRegion']==1, 1, 0)
df['CitizenRegion2'] = np.where(df['CitizenRegion']==2, 1, 0)
df['CitizenRegion3'] = np.where(df['CitizenRegion']==3, 1, 0)
df['CitizenRegion4'] = np.where(df['CitizenRegion']==4, 1, 0)
df['CitizenRegion5'] = np.where(df['CitizenRegion']==5, 1, 0)
df['CitizenRegion6'] = np.where(df['CitizenRegion']==6, 1, 0)
df['CitizenRegion7'] = np.where(df['CitizenRegion']==7, 1, 0)
df['CitizenRegion8'] = np.where(df['CitizenRegion']==8, 1, 0)
df['CitizenRegion9'] = np.where(df['CitizenRegion']==9, 1, 0)
df['CitizenRegion10'] = np.where(df['CitizenRegion']==10, 1, 0)
df['CitizenRegion11'] = np.where(df['CitizenRegion']==11, 1, 0)
df['CitizenRegion12'] = np.where(df['CitizenRegion']==12, 1, 0)
df['CitizenRegion13'] = np.where(df['CitizenRegion']==13, 1, 0)
df['CitizenRegion14'] = np.where(df['CitizenRegion']==14, 1, 0)
df['CitizenRegion15'] = np.where(df['CitizenRegion']==15, 1, 0)
df['CitizenRegion16'] = np.where(df['CitizenRegion']==16, 1, 0)
df['CitizenRegion17'] = np.where(df['CitizenRegion']==17, 1, 0)
df['CitizenRegion18'] = np.where(df['CitizenRegion']==18, 1, 0)
df['CitizenRegion19'] = np.where(df['CitizenRegion']==19, 1, 0)
df['CitizenRegion20'] = np.where(df['CitizenRegion']==20, 1, 0)
df['CitizenRegion21'] = np.where(df['CitizenRegion']==21, 1, 0)
df['CitizenRegion22'] = np.where(df['CitizenRegion']==22, 1, 0)
df['CitizenRegion23'] = np.where(df['CitizenRegion']==23, 1, 0)
df['postalValue0'] = np.where(df['postalValue']==0, 1, 0)
df['postalValue1'] = np.where(df['postalValue']==1, 1, 0)
df['postalValue2'] = np.where(df['postalValue']==2, 1, 0)
df['postalValue3'] = np.where(df['postalValue']==3, 1, 0)
df['postalValue4'] = np.where(df['postalValue']==4, 1, 0)
df['postalValue5'] = np.where(df['postalValue']==5, 1, 0)
df['postalValue6'] = np.where(df['postalValue']==6, 1, 0)
df['postalValue7'] = np.where(df['postalValue']==7, 1, 0)
df['postalValue8'] = np.where(df['postalValue']==8, 1, 0)
df['postalValue9'] = np.where(df['postalValue']==9, 1, 0)
df['Ind_Code11'] = np.where(df['Ind_code']==11, 1, 0)
df['Ind_Code13'] = np.where(df['Ind_code']==13, 1, 0)
df['Ind_Code15'] = np.where(df['Ind_code']==15, 1, 0)
df['Ind_Code17'] = np.where(df['Ind_code']==17, 1, 0)
df['Ind_Code19'] = np.where(df['Ind_code']==19, 1, 0)
df['Ind_Code20'] = np.where(df['Ind_code']==20, 1, 0)
df['Ind_Code21'] = np.where(df['Ind_code']==21, 1, 0)
df['Ind_Code23'] = np.where(df['Ind_code']==23, 1, 0)
df['Ind_Code25'] = np.where(df['Ind_code']==25, 1, 0)
df['Ind_Code27'] = np.where(df['Ind_code']==27, 1, 0)
df['Ind_Code29'] = np.where(df['Ind_code']==29, 1, 0)
df['Ind_Code30'] = np.where(df['Ind_code']==30, 1, 0)
df['Ind_Code31'] = np.where(df['Ind_code']==31, 1, 0)
df['Ind_Code33'] = np.where(df['Ind_code']==33, 1, 0)
df['Ind_Code35'] = np.where(df['Ind_code']==35, 1, 0)
df['Ind_Code37'] = np.where(df['Ind_code']==37, 1, 0)
df['Ind_Code39'] = np.where(df['Ind_code']==39, 1, 0)
df['Ind_Code41'] = np.where(df['Ind_code']==41, 1, 0)
df['Ind_Code43'] = np.where(df['Ind_code']==43, 1, 0)
df['Ind_Code45'] = np.where(df['Ind_code']==45, 1, 0)
df['Ind_Code47'] = np.where(df['Ind_code']==47, 1, 0)
df['Ind_Code49'] = np.where(df['Ind_code']==49, 1, 0)
df['Ind_Code51'] = np.where(df['Ind_code']==51, 1, 0)
df['Ind_Code53'] = np.where(df['Ind_code']==53, 1, 0)
df['Ind_Code90'] = np.where(df['Ind_code']==90, 1, 0)
df['Ind_Code91'] = np.where(df['Ind_code']==91, 1, 0)
df['upperWage'] = np.where(df['Year_wage_offer']>=150000, 1, 0)
df['upperMiddleWage'] = np.where((df['Year_wage_offer']>=100000) & (df['Year_wage_offer']<=150000), 1, 0)
df['middleWage'] = np.where((df['Year_wage_offer']>=30000) & (df['Year_wage_offer']<=99999), 1, 0)
df['povertyWage'] = np.where(df['Year_wage_offer']<=30000, 1, 0)

In [108]:
df.loc[:,'EMPLOYER_YR_ESTAB'] *= .001

In [109]:
list(df)

['CASE_STATUS',
 'FOREIGN_WORKER_INFO_EDUCATION',
 'JOB_INFO_JOB_REQ_NORMAL',
 'CitizenRegion',
 'postalValue',
 'DEC_YEAR',
 'LAWYER',
 'JOB_INFO_FOREIGN_LANG_REQ',
 'JOB_INFO_COMBO_OCCUPATION',
 'Wage_level',
 'Year_wage_offer',
 'Year_wage_pw',
 'Ind_code',
 'Higher education',
 'Lower education',
 'Total_rec',
 'EMPLOYER_YR_ESTAB',
 'H1B_VISA',
 'Other_Visa',
 'No_US_Visa',
 'Same_education',
 'PW_outlier',
 'Offer_outlier',
 'EducationNA',
 'Education1',
 'Education2',
 'Education3',
 'Education4',
 'Education5',
 'Education6',
 'CitizenRegion1',
 'CitizenRegion2',
 'CitizenRegion3',
 'CitizenRegion4',
 'CitizenRegion5',
 'CitizenRegion6',
 'CitizenRegion7',
 'CitizenRegion8',
 'CitizenRegion9',
 'CitizenRegion10',
 'CitizenRegion11',
 'CitizenRegion12',
 'CitizenRegion13',
 'CitizenRegion14',
 'CitizenRegion15',
 'CitizenRegion16',
 'CitizenRegion17',
 'CitizenRegion18',
 'CitizenRegion19',
 'CitizenRegion20',
 'CitizenRegion21',
 'CitizenRegion22',
 'CitizenRegion23',
 'postalVa

In [110]:
df.dropna(axis=0, inplace=True)
df.astype(int,inplace = True)
df = df.drop(['Higher education', 'Lower education','Other_Visa','No_US_Visa','PW_outlier','Offer_outlier','CitizenRegion','postalValue','FOREIGN_WORKER_INFO_EDUCATION','DEC_YEAR','Wage_level','Year_wage_offer','Year_wage_pw','Ind_code', 'Same_education'], axis=1)

In [111]:
X = df.iloc[:,1:]
Y = df.iloc[:,0]
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=0)
classifier = LogisticRegression()
classifier.fit(X_train, y_train)
coef = classifier.coef_[0]
print (coef)
y_pred = classifier.predict(X_test)
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)
print('Accuracy of logistic regression classifier on test set: {:.5f}\n'.format(classifier.score(X_test, y_test)))
print('Presicion: {:.5f}\n'.format(sklearn.metrics.precision_score(y_test,y_pred)))
print('Recall: {:.5f}\n'.format(sklearn.metrics.recall_score(y_test,y_pred)))
print('F1: {:.5f}\n'.format(sklearn.metrics.f1_score(y_test,y_pred)))

[ -2.09443005e-01   1.79562265e-01  -2.16175743e-01  -3.73639966e-01
  -1.41636102e-05   3.43955971e-01   8.05842160e-02   0.00000000e+00
   7.67130650e-02  -4.52077773e-02  -9.41832080e-02   2.22104543e-01
   2.02984814e-01   6.10445770e-01   2.36975787e-01  -1.76204827e-01
   1.48100950e-01   2.54029811e-01  -2.79508104e-02   1.58835331e-01
  -1.43956229e-01  -1.68003795e-01  -3.58100283e-02   1.26267558e-01
  -5.34748972e-03   2.14425407e-01   1.39349070e-01  -9.56522173e-02
   1.87360425e-01   7.57537042e-02   1.53911277e-01   1.71075154e-01
   9.31880382e-02  -4.74543633e-02  -9.10934027e-02  -6.51662948e-02
  -1.29775848e-01   2.33085666e-01   2.21030266e-01   2.32666297e-01
   2.26409371e-01   2.32719299e-01   4.81440566e-01   2.67758413e-01
   3.25000214e-01   1.20418790e-01   7.44546916e-02   3.46684582e-01
   4.50331009e-01   4.87609344e-01   4.48123935e-01   3.81184619e-01
   2.93606073e-01   3.40728884e-01   3.21891645e-01   4.96908818e-02
   3.59557837e-01   4.07583532e-01

In [112]:
sampled_df =df.copy() 

In [113]:
df.shape

(297601, 78)

In [114]:
np.random.seed(11)
ones = sampled_df[sampled_df['CASE_STATUS'] == 1]
remove_n = (.66 * ones.count()).astype(int)
remove_n = remove_n['CASE_STATUS']
drop_indices = np.random.choice(ones.index, remove_n, replace=False)
sampled_df = ones.drop(drop_indices)
zeros = df[df['CASE_STATUS'] == 0]
frames = [zeros,sampled_df]
sampled_df = pd.concat(frames)

In [115]:
sampled_df['CASE_STATUS'].value_counts()

1    20007
0    19276
Name: CASE_STATUS, dtype: int64

In [116]:
X = sampled_df.iloc[:,1:]
Y = sampled_df.iloc[:,0]
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=0)

In [117]:
#fig, ax = plt.subplots(figsize=(25,25)) 
#sns.heatmap(df.corr(), annot = df.corr())
#plt.show()

In [118]:
classifier = LogisticRegression()
classifier.fit(X_train, y_train)
coef = classifier.coef_[0]
print (coef)
y_pred = classifier.predict(X_test)
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)
print('Accuracy of logistic regression classifier on test set: {:.5f}\n'.format(classifier.score(X_test, y_test)))
print('Presicion: {:.5f}\n'.format(sklearn.metrics.precision_score(y_test,y_pred)))
print('Recall: {:.5f}\n'.format(sklearn.metrics.recall_score(y_test,y_pred)))
print('F1: {:.5f}\n'.format(sklearn.metrics.f1_score(y_test,y_pred)))

[ -2.12830170e-01   1.82438732e-01  -2.28149388e-01  -1.21209009e-01
  -6.14904559e-07   3.05272735e-01   8.79870337e-03   0.00000000e+00
  -3.13927112e-01  -4.54932156e-01  -3.30286987e-01  -1.66331786e-01
  -1.76669366e-01   2.09885144e-01   1.23784782e-01  -2.19846358e-01
  -1.80699338e-01   5.11676737e-02   2.55974183e-02   9.51152328e-02
  -5.83848985e-02  -3.16691628e-01  -3.20534879e-01   8.98374499e-05
  -7.38805987e-02   4.25020482e-02   5.48414254e-02  -1.78581120e-01
  -6.49628159e-02  -4.02283825e-02   7.29734563e-03   1.63454301e-01
  -7.80894534e-02  -7.29900656e-02   3.62668965e-01  -2.14375646e-01
  -3.39516110e-01   2.99930411e-01   2.95867808e-01   3.31342549e-01
   3.17761253e-01   3.64142468e-01   5.59894824e-01   3.90767739e-01
   4.66645453e-01   2.71247809e-01   1.71260641e-01   6.45241239e-01
   6.79177533e-01   6.84627000e-01   5.96968639e-01   5.99616662e-01
   2.81605006e-02   2.86256635e-01   8.22218039e-01   2.94847445e-01
   6.43513086e-01   4.92007088e-01

In [20]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
names = list(df)
 
#use linear regression as the model
lr = LogisticRegression()
#rank all features, i.e continue the elimination until the last one
rfe = RFE(lr, n_features_to_select=15)
rfe.fit(X,Y)
 
print("Features sorted by their rank:")
print(sorted(zip(map(lambda x: round(x, 4), rfe.ranking_), names)))

Features sorted by their rank:
[(1, 'CitizenRegion7'), (1, 'CitizenRegion8'), (1, 'Ind_Code11'), (1, 'Ind_Code13'), (1, 'Ind_Code15'), (1, 'Ind_Code17'), (1, 'Ind_Code19'), (1, 'Ind_Code21'), (1, 'Ind_Code31'), (1, 'Ind_Code33'), (1, 'Ind_Code39'), (1, 'Ind_Code43'), (1, 'Ind_Code51'), (1, 'Ind_Code53'), (1, 'postalValue9'), (2, 'Education5'), (3, 'Ind_Code27'), (4, 'Ind_Code25'), (5, 'Education2'), (6, 'CitizenRegion6'), (7, 'Ind_Code30'), (8, 'CitizenRegion13'), (9, 'postalValue4'), (10, 'CitizenRegion4'), (11, 'CASE_STATUS'), (12, 'Ind_Code41'), (13, 'Ind_Code35'), (14, 'Ind_Code45'), (15, 'Ind_Code49'), (16, 'Ind_Code20'), (17, 'Ind_Code23'), (18, 'Ind_Code47'), (19, 'Ind_Code37'), (20, 'Education1'), (21, 'CitizenRegion22'), (22, 'CitizenRegion18'), (23, 'LAWYER'), (24, 'CitizenRegion19'), (25, 'CitizenRegion17'), (26, 'postalValue6'), (27, 'upperMiddleWage'), (28, 'JOB_INFO_JOB_REQ_NORMAL'), (29, 'Ind_Code29'), (30, 'EducationNA'), (31, 'CitizenRegion10'), (32, 'CitizenRegion2'),

In [21]:
import statsmodels.api as sm
logit = sm.Logit(y_train, X_train)
result = logit.fit()
result.summary()

C:\Anaconda\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


         Current function value: 0.643695
         Iterations: 35


LinAlgError: Singular matrix